In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch

In [2]:
from src.mm_v1.interface import testkernel, copykernel, mmkernelv1

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Creating extension directory /home/max/.cache/torch_extensions/py311_cu118/mm_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/mm_v1/build.ninja...
Building extension module mm_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=mm_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/mm_v1/interface.cc -o interface.o 
[2/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/envs/

Loading extension module mm_v1...


In [3]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [4]:
S = 8 # sequence length
DH = 8 # hidden size

In [5]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH)) / 10.
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[ 0.0000,  0.1001,  0.2002,  0.3008,  0.4004,  0.5000,  0.6016,  0.6992],
         [ 0.8008,  0.8984,  1.0000,  1.1016,  1.2031,  1.2969,  1.3984,  1.5000],
         [ 1.6016,  1.7031,  1.7969,  1.8984,  2.0000,  2.0938,  2.2031,  2.2969],
         [ 2.4062,  2.5000,  2.5938,  2.7031,  2.7969,  2.9062,  3.0000,  3.0938],
         [ 3.2031,  3.2969,  3.4062,  3.5000,  3.5938,  3.7031,  3.7969,  3.9062],
         [ 4.0000,  4.0938,  4.1875,  4.3125,  4.4062,  4.5000,  4.5938,  4.6875],
         [ 4.8125,  4.9062,  5.0000,  5.0938,  5.1875,  5.3125,  5.4062,  5.5000],
         [ 5.5938,  5.6875,  5.8125,  5.9062,  6.0000,  6.0938,  6.1875,  6.3125],
         [ 6.4062,  6.5000,  6.5938,  6.6875,  6.8125,  6.9062,  7.0000,  7.0938],
         [ 7.1875,  7.3125,  7.4062,  7.5000,  7.5938,  7.6875,  7.8125,  7.9062],
         [ 8.0000,  8.1250,  8.1875,  8.3125,  8.3750,  8.5000,  8.6250,  8.6875],
         [ 8.8125,  8.8750,  9.0000,  9.1250

In [6]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969],
         [ 9.1875,  9.1875,  9.1875,  9.1875,  9.1875,  9.1875,  9.1875,  9.1875],
         [15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250],
         [22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000],
         [28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750],
         [34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500],
         [41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500],
         [47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000],
         [54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000],
         [60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000],
         [67.0000, 67.0000, 67.0000, 67.0000, 67.0000, 67.0000, 67.0000, 67.0000],
         [73.0000, 73.0000, 73.0000, 73.0000, 73.0000, 73.0000, 73.0000, 73.0000],
    

In [7]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [8]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 16, n: 8, k: 8
blocksxy: 2-4, threads: 4-4
In Kernel: m: 16, n: 8, k: 8
<1, 0> - (ty,i) - As: (b) 7.187500
<1, 0> - (ty,i) - Bs: (b) 1.000000
In Kernel: gdim.x: 2, gdim.y: 4, bdim.x: 4, bdim.y: 4
<0, 9> - (cx,cy)-InLoop:Csub: (b) 7.187500
<1, 1> - (ty,i) - As: (b) 7.312500
<1, 1> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 14.500000
<1, 2> - (ty,i) - As: (b) 7.406250
<1, 2> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 21.875000
<1, 3> - (ty,i) - As: (b) 7.500000
<1, 3> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 29.375000
<0, 9> - (cx,cy)-AfterLoop:Csub: (b) 29.375000
<1, 0> - (ty,i) - As: (b) 7.593750
<1, 0> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 37.000000
<1, 1> - (ty,i) - As: (b) 7.687500
<1, 1> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 44.750000
<1, 2> - (ty,i) - As: (b) 7.812500
<1, 2> - (ty,i) - Bs: (b) 1.000000
<0, 9> - (cx,cy)-InLoop:Csub: (b) 52

(tensor([[ 2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969,  2.7969],
         [ 9.2500,  9.2500,  9.2500,  9.2500,  9.2500,  9.2500,  9.2500,  9.2500],
         [15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250, 15.6250],
         [22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000, 22.0000],
         [28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750, 28.3750],
         [34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500, 34.7500],
         [41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500, 41.2500],
         [47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000, 47.5000],
         [54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000, 54.0000],
         [60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000, 60.5000],
         [66.5000, 66.5000, 66.5000, 66.5000, 66.5000, 66.5000, 66.5000, 66.5000],
         [73.5000, 73.5000, 73.5000, 73.5000, 73.5000, 73.5000, 73.5000, 73.5000],
    

In [9]:
matA[9]

tensor([7.1875, 7.3125, 7.4062, 7.5000, 7.5938, 7.6875, 7.8125, 7.9062],
       device='cuda:0', dtype=torch.bfloat16)

In [57]:
matA[9].cumsum(-1)

tensor([ 7.1875, 14.5000, 21.8750, 29.3750, 37.0000, 44.7500, 52.5000, 60.5000],
       device='cuda:0', dtype=torch.bfloat16)

In [58]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 448., 528., 608.], device='cuda:0',
       dtype=torch.bfloat16)

In [59]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 448., 524., 604.], dtype=torch.bfloat16)

In [60]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 447., 525., 604.])

In [61]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 447., 525., 604.], device='cuda:0',
       dtype=torch.float16)

In [62]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [63]:
# mat @ mat.T @ mat

### pytorch version

In [64]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [65]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))